# Synthetic Traces Plotting/Analysis

This notebook explores the input rate generation methods implemented in the `dfaas_input_rate' module. The generation is done in traces, one for each agent, at the beginning of the episode for all steps. There are two main types of traces:

* Traces generated from real data: not covered in this notebook.
* Traces generated from synthetic data: covered in this notebook.

Some constraints (or assumptions):

1. Each episode has a fixed length of 288 steps (each step represents 5 minutes in a 24-hour period).
2. For each step and agent, the input rate takes positive values in a fixed range (1, 150).

In [ ]:
# Common imports.
from pathlib import Path

%matplotlib widget
import base

import dfaas_input_rate

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import ipywidgets

## Sinusoidal traces

## Generation method (current)

In [ ]:
rng = np.random.default_rng(seed=42)  # RNG used to generate the requests.

In [ ]:
def plot_single_trace(base_rng):
    # These are just fixed values for one trace.
    agents = ["node_0"]
    max_steps, min_reqs, max_reqs = 288, 0, 150
    limits = {"node_0": {"min": min_reqs, "max": max_reqs}}

    # Generate a specific seed for this trace.
    iinfo = np.iinfo(np.uint32)
    seed = rng.integers(0, high=iinfo.max, size=1)[0].item()
    specific_rng = np.random.default_rng(seed=seed)

    trace = dfaas_input_rate.synthetic_sinusoidal(max_steps, agents, limits, specific_rng)

    plt.close(fig="single_trace_fixed_args")
    fig = plt.figure(num="single_trace_fixed_args", layout="constrained")
    fig.canvas.header_visible = False
    ax = fig.subplots()

    ax.plot(trace["node_0"])

    # ax.set_title(f"Function invocations ({seed = })")
    ax.set_ylabel("Invocations")
    ax.yaxis.set_major_locator(ticker.MultipleLocator(10))
    ax.set_ylim(bottom=min_reqs, top=max_reqs + 5)
    ax.set_xlabel("Step")
    ax.xaxis.set_major_locator(ticker.MultipleLocator(25))

    ax.grid(axis="both")
    ax.set_axisbelow(True)

    return fig


fig = plot_single_trace(rng)

## Generation method (next)

In [ ]:
rng = np.random.default_rng(seed=42)  # RNG used to generate the requests.
min_reqs, max_reqs = 0, 150  # Range of the generated requests.
max_steps = 288  # Steps of a single episode (see the DFaaS env).

In [ ]:
# Generates the requests for a single agent.
def synthetic_sinusoidal(
    average_requests=74,
    amplitude_requests=72,
    noise_ratio=0.1,
    function=None,
    seed=0,
):
    steps = np.arange(max_steps)

    if function is None:
        # Must generate a different RNG to do not alter the RNG
        # used to sample.
        fn_rng = np.random.default_rng(seed=seed)
        function = fn_rng.choice([np.sin, np.cos])

    rng = np.random.default_rng(seed=seed)
    average_requests = np.clip(rng.normal(loc=70), 60, 80)

    # Sample a period with a fixed interval.
    period = rng.uniform(5, high=30)

    # Sample the function.
    base_input = average_requests + amplitude_requests * function(steps / period)

    # Add some noise.
    noisy_input = base_input + noise_ratio * rng.normal(0, amplitude_requests, size=max_steps)
    requests = np.asarray(noisy_input, dtype=np.int32)

    # Clip the excess values respecting the minimum and maximum values
    # for the input requests observation.
    np.clip(requests, min_reqs, max_reqs, out=requests)

    return requests

### Plotting of a generic trace

In [ ]:
def plot_sinusoidal_trace():
    avg_reqs = ipywidgets.IntSlider(
        value=74,
        min=0,
        max=200,
        description="Average requests [0, 200]:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )
    ampl_reqs = ipywidgets.IntSlider(
        value=60,
        min=0,
        max=200,
        description="Amplitude requests [0, 200]:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )
    noise = ipywidgets.FloatSlider(
        value=0.1,
        min=0,
        max=2,
        description="Noise [0, 2]:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )
    function = ipywidgets.Dropdown(
        options=[("None", None), ("sin", np.sin), ("cos", np.cos)],
        index=0,
        description="Function:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )
    seed = ipywidgets.BoundedIntText(
        value=42,
        min=0,
        max=np.iinfo(np.uint32).max,
        description="Seed:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )

    with plt.ioff():
        fig_invocs = plt.figure(layout="constrained")
        fig_dist = plt.figure(layout="constrained")
    fig_invocs.canvas.header_visible = False
    ax_invocs = fig_invocs.subplots()
    fig_dist.canvas.header_visible = False
    ax_dist = fig_dist.subplots()

    def make_plot():
        # Generate a new trace.
        trace = synthetic_sinusoidal(
            average_requests=avg_reqs.value,
            amplitude_requests=ampl_reqs.value,
            noise_ratio=noise.value,
            function=function.value,
            seed=seed.value,
        )

        # Make the invocations plot.
        ax_invocs.clear()
        ax_invocs.plot(trace)

        ax_invocs.set_title(f"Function invocations")
        ax_invocs.set_ylabel("Invocations")
        ax_invocs.yaxis.set_major_locator(ticker.MultipleLocator(10))
        ax_invocs.set_ylim(bottom=min_reqs, top=max_reqs + 5)
        ax_invocs.set_xlabel("Step")
        ax_invocs.xaxis.set_major_locator(ticker.MultipleLocator(25))

        ax_invocs.grid(axis="both")
        ax_invocs.set_axisbelow(True)

        # Make the invocations distribution plot.
        ax_dist.clear()

        # The X-axis (freqs_idx) and the values (freqs) must have the same length.
        freqs = pd.Series(trace).value_counts()
        freqs_idx = np.arange(freqs.index.max() + 1)
        freqs = freqs.reindex(range(freqs.index.max() + 1), fill_value=0)
        ax_dist.bar(freqs_idx, freqs)

        ax_dist.set_title(f"Function invocations distribution")
        ax_dist.set_ylabel("Frequency")
        ax_dist.set_xlabel("Invocation")
        # Force the X-axis to have integer values (float by default).
        ax_dist.xaxis.get_major_locator().set_params(integer=True)

        ax_dist.grid(axis="both")
        ax_dist.set_axisbelow(True)

        fig_dist.canvas.draw_idle()
        fig_invocs.canvas.draw_idle()
        fig_invocs.canvas.flush_events()
        fig_dist.canvas.flush_events()

    # Make the initial plot with the default values.
    make_plot()

    # Link the input widgets to the plotting function.
    avg_reqs.observe(lambda change: make_plot(), names="value")
    ampl_reqs.observe(lambda change: make_plot(), names="value")
    noise.observe(lambda change: make_plot(), names="value")
    function.observe(lambda change: make_plot(), names="value")
    seed.observe(lambda change: make_plot(), names="value")

    # Callback when the user press the reset button.
    def reset(widget, default_value):
        widget.index = 0
        make_plot()

    # Create the reset buttons and link them to the correct widget.
    avg_reqs_reset = ipywidgets.Button(description="Reset", icon="undo")
    avg_reqs_reset.on_click(lambda _: reset(avg_reqs, 76))
    ampl_reqs_reset = ipywidgets.Button(description="Reset", icon="undo")
    ampl_reqs_reset.on_click(lambda _: reset(ampl_reqs, 60))
    noise_reset = ipywidgets.Button(description="Reset", icon="undo")
    noise_reset.on_click(lambda _: reset(noise, 0.1))
    function_reset = ipywidgets.Button(description="Reset", icon="undo")
    function_reset.on_click(lambda _: reset(function, 0))

    avg_reqs_reset.click()

    # Create the last button to generate a random seed.
    def random_seed():
        iinfo = np.iinfo(np.uint32)
        seed.value = rng.integers(0, high=iinfo.max, size=1)[0].item()

    random_seed_btn = ipywidgets.Button(description="Random", icon="shuffle")
    random_seed_btn.on_click(lambda _: random_seed())

    inputs = ipywidgets.VBox(
        [
            ipywidgets.HBox([avg_reqs, avg_reqs_reset]),
            ipywidgets.HBox([ampl_reqs, ampl_reqs_reset]),
            ipywidgets.HBox([noise, noise_reset]),
            ipywidgets.HBox([function, function_reset]),
            ipywidgets.HBox([seed, random_seed_btn]),
        ]
    )

    return ipywidgets.AppLayout(
        header=inputs,
        center=ipywidgets.VBox([fig_invocs.canvas, fig_dist.canvas]),
        pane_heights=[0.6, 5, 0],
    )


plot_sinusoidal_trace()

## Compare multiple traces

In [ ]:
def make_traces(n_traces):
    assert n_traces > 0

    rng = np.random.default_rng(seed=42)

    # Fixed values for all traces.
    agents = ["node_0"]
    max_steps, min_reqs, max_reqs = 288, 0, 150
    limits = {"node_0": {"min": min_reqs, "max": max_reqs}}

    # Generate a specific seed for this trace.
    iinfo = np.iinfo(np.uint32)
    seed = rng.integers(0, high=iinfo.max, size=1)[0].item()
    specific_rng = np.random.default_rng(seed=seed)

    traces = []
    for i in range(n_traces):
        trace = dfaas_input_rate.synthetic_sinusoidal(max_steps, agents, limits, specific_rng)
        traces.append(trace["node_0"])
    return traces


n_traces = 10
traces = make_traces(n_traces)

In [ ]:
traces_next = []

rng = np.random.default_rng(seed=42)
for i in range(n_traces):
    traces_next.append(synthetic_sinusoidal(seed=rng.integers(low=0, high=20000)))

### Dynamic Time Warping (DTW)

See: https://dynamictimewarping.github.io/python/

In [ ]:
from dtw import dtw

In [ ]:
def compute_dtw(traces):
    res_dtw, res_dtw_norm = [], []

    for x_index in range(len(traces)):
        x_dtw, x_dtw_norm = [], []
        for y_index in range(len(traces)):
            if x_index == y_index:
                continue

            x = traces[x_index]
            y = traces[y_index]
            dtw_xy = dtw(x, y)

            x_dtw.append(dtw_xy.distance)
            x_dtw_norm.append(dtw_xy.normalizedDistance)

        res_dtw.append(np.mean(x_dtw))
        res_dtw_norm.append(np.mean(x_dtw_norm))

    return np.mean(res_dtw), np.mean(res_dtw_norm)


dtw_distance, dtw_distance_norm = compute_dtw(traces)
dtw_distance_next, dtw_distance_next_norm = compute_dtw(traces_next)

print("Current traces:")
print("  Dynamic Time Warping distance (average on all traces) =", dtw_distance)
print("  Dynamic Time Warping normalized (average on all traces) =", dtw_distance_norm)
print()
print("Next traces:")
print("  Dynamic Time Warping distance (average on all traces) =", dtw_distance_next)
print("  Dynamic Time Warping normalized (average on all traces) =", dtw_distance_next_norm)

In [ ]:
def dtwPlotTwoWay(
    d,
    fig,
    xts=None,
    yts=None,
    offset=0,
    ts_type="l",
    match_indices=None,
    match_col="gray",
    xlab="Step",
    ylab="Invocations",
    **kwargs,
):
    """Customized version of dtwPlotTwoWay function for this notebook.

    Original code: https://github.com/DynamicTimeWarping/dtw-python/blob/master/dtw/dtwPlot.py
    """
    import matplotlib.pyplot as plt
    from matplotlib import collections as mc

    if xts is None or yts is None:
        try:
            xts = d.query
            yts = d.reference
        except:
            raise ValueError("Original timeseries are required")

    # ytso = yts + offset
    offset = -offset

    xtimes = np.arange(len(xts))
    ytimes = np.arange(len(yts))

    ax = fig.subplots()

    ax.set_xlabel(xlab)
    ax.set_ylabel(ylab)

    ax.plot(xtimes, np.array(xts), color="k", label="X", **kwargs)
    ax.plot(ytimes, np.array(yts) - offset, label="Y", **kwargs)  # Plot with offset applied
    fig.legend()

    if offset != 0:
        # Create an offset axis
        ax2 = ax.twinx()
        ax2.tick_params("y", colors="b")
        ql, qh = ax.get_ylim()
        ax2.set_ylim(ql + offset, qh + offset)

    # https://stackoverflow.com/questions/21352580/matplotlib-plotting-numerous-disconnected-line-segments-with-different-colors
    if match_indices is None:
        idx = np.linspace(0, len(d.index1) - 1)
    elif not hasattr(match_indices, "__len__"):
        idx = np.linspace(0, len(d.index1) - 1, num=match_indices)
    else:
        idx = match_indices
    idx = np.array(idx).astype(int)

    col = []
    for i in idx:
        col.append([(d.index1[i], xts[d.index1[i]]), (d.index2[i], -offset + yts[d.index2[i]])])

    lc = mc.LineCollection(col, linewidths=1, linestyles=":", colors=match_col)
    ax.add_collection(lc)

    return ax


def make_dtw_plot(traces):
    n = len(traces)

    x_index = ipywidgets.BoundedIntText(
        value=0,
        min=0,
        max=len(traces),
        description=f"X trace [0, {n-1}]:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )
    y_index = ipywidgets.BoundedIntText(
        value=1,
        min=0,
        max=len(traces),
        description=f"Y trace [0, {n-1}]:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )

    with plt.ioff():
        plt.close(fig="make_dtw_plot")
        fig = plt.figure(num="make_dtw_plot", layout="constrained")
    fig.canvas.header_visible = False

    out = ipywidgets.Output()

    def make_plot():
        x = traces[x_index.value]
        y = traces[y_index.value]

        dtw_xy = dtw(x, y, keep_internals=True)

        with out:
            out.clear_output()
            print(f"Distance: {dtw_xy.distance:.3f}")
            print(f"Distance normalized: {dtw_xy.normalizedDistance:.3f}")

        # Make the plot.
        fig.clear()

        dtwPlotTwoWay(dtw_xy, fig)

        fig.canvas.draw_idle()
        fig.canvas.draw_idle()

    # Make the initial plot with the default values.
    make_plot()

    # Link the input widgets to the plotting function.
    x_index.observe(lambda change: make_plot(), names="value")
    y_index.observe(lambda change: make_plot(), names="value")

    inputs = ipywidgets.VBox([x_index, y_index])

    return ipywidgets.AppLayout(
        header=ipywidgets.VBox([inputs, out]),
        center=fig.canvas,
        pane_heights=[0.9, 5, 0],
    )


make_dtw_plot(traces)

### Basic Statistical Measures

#### Mean Absolute Error (MAE)

In [ ]:
def compute_single_mae(x, y):
    assert len(x) == len(y)
    n = len(x)

    return (1 / n) * np.sum(np.abs(x - y))


def compute_mae(traces):
    mae = []

    for x_index in range(len(traces)):
        x_mae = []
        for y_index in range(len(traces)):
            if x_index == y_index:
                continue

            x = traces[x_index]
            y = traces[y_index]
            mae_xy = compute_single_mae(x, y)

            x_mae.append(mae_xy)

        mae.append(np.mean(x_mae))

    return np.mean(mae)


mae = compute_mae(traces)

print(f"Mean Absolute Error (average on all traces) = {mae:.3f}")

#### Mean Squared Error (MSE)

In [ ]:
def compute_single_mse(x, y):
    assert len(x) == len(y)
    n = len(x)

    return (1 / n) * np.sum(np.square(x - y))


def compute_mse(traces):
    mse = []

    for x_index in range(len(traces)):
        x_mse = []
        for y_index in range(len(traces)):
            if x_index == y_index:
                continue

            x = traces[x_index]
            y = traces[y_index]
            mse_xy = compute_single_mse(x, y)

            x_mse.append(mse_xy)

        mse.append(np.mean(x_mse))

    return np.mean(mse)


mse = compute_mse(traces)

print(f"Mean Square Error (average on all traces) = {mse:.3f}")

#### Root Mean Squared Error (RMSE)

In [ ]:
def compute_single_rmse(x, y):
    mse = compute_single_mse(x, y)
    return np.sqrt(mse)


def compute_rmse(traces):
    mse = compute_mse(traces)
    return np.sqrt(mse)


rmse = compute_rmse(traces)

print(f"Root Mean Square Error (average on all traces) = {rmse:.3f}")

#### Summary

In [ ]:
def make_basic_statistical_measures_app(traces):
    n = len(traces)

    x_index = ipywidgets.BoundedIntText(
        value=0,
        min=0,
        max=len(traces),
        description=f"X trace [0, {n-1}]:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )
    y_index = ipywidgets.BoundedIntText(
        value=1,
        min=0,
        max=len(traces),
        description=f"Y trace [0, {n-1}]:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )

    out = ipywidgets.Output()

    def make_widget():
        x = traces[x_index.value]
        y = traces[y_index.value]

        mae = compute_single_mae(x, y)
        mse = compute_single_mse(x, y)
        rmse = compute_single_rmse(x, y)

        with out:
            out.clear_output()
            print(f"Mean Absolute Error: {mae:.3f}")
            print(f"Mean Square Error: {mse:.3f}")
            print(f"Root Mean Square Error: {rmse:.3f}")

    # Make the initial plot with the default values.
    make_widget()

    # Link the input widgets to the plotting function.
    x_index.observe(lambda change: make_widget(), names="value")
    y_index.observe(lambda change: make_widget(), names="value")

    inputs = ipywidgets.VBox([x_index, y_index])

    return ipywidgets.AppLayout(header=inputs, center=out)


make_basic_statistical_measures_app(traces)